In [1]:
import numpy as np

from collections import defaultdict
import copy

import sys
import os
import time
import datetime
import torch
import torchvision

import matplotlib.pyplot as plt
import torch.nn as nn

/home/grace/anaconda3/envs/mypy3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/grace/anaconda3/envs/mypy3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
from collections import OrderedDict  # Import OrderedDict

def get_model2(model_name, dataset_name, pretrained=False, load_model=False):
    """Retrieve an appropriate architecture."""
    if 'CIFAR' in dataset_name or 'MNIST' in dataset_name:
        if pretrained:
            raise ValueError('Loading pretrained models is only supported for ImageNet.')
        in_channels = 1 if dataset_name == 'MNIST' else 3
        num_classes = 10 if dataset_name in ['CIFAR10', 'MNIST'] else 100
        if 'ResNet' in model_name:
            model = resnet_picker(model_name, dataset_name)
            if load_model:
                # Load the state_dict from the file
                state_dict = torch.load(load_model)
                
                # Handle cases where the state_dict is stored under a specific key
                if 'state_dict' in state_dict:
                    state_dict = state_dict['state_dict']

                # Adapt keys if they include 'module.' prefix (e.g., from DataParallel)
                new_state_dict = OrderedDict()
                for k, v in state_dict.items():
                    if k.startswith('module.'):
                        name = k[7:]  # Remove 'module.' prefix
                    else:
                        name = k
                    new_state_dict[name] = v
                
                # Load the adapted state_dict
                model.load_state_dict(new_state_dict, strict=False)  # Use strict=False to allow flexibility
                return model
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")

In [4]:
import sys
sys.path.append('/home/grace/demo/poison/ASPARaGUS')

from forest.victims.models import get_model
from forest.victims.models import resnet_picker
#load pretrained model
#model = torchvision.models.resnet18(pretrained=True)
model = get_model('ResNet18', 'CIFAR10', load_model="/home/grace/PoisonTest/models/poisoned_model/full_epoch_30.pth")
#model = get_model('ResNet18', 'CIFAR10', load_model="/home/grace/demo/poison/models/clean_model/full_epoch_30.pth")

num_ftrs = model.fc.in_features

#modify output layer for 2 classes - for this case the classes are regular frogs and noisy frogs
#but for the project it would be poisoned vs non-poisoned images
model.fc = torch.nn.Linear(num_ftrs, 2)
print(model.fc.out_features)


2


In [5]:
# Set the model to evaluation mode
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layers): Sequential(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3,

In [6]:
import torchvision.transforms as transforms

# Define the transform for preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Move the model to the appropriate device (GPU/CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layers): Sequential(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3,

In [7]:
import os
import torch
from PIL import Image  # Importing Image module
from torchvision import transforms

# Path to the folder containing images
allfrog_folder = "/home/grace/allfrog"

# Create an output file to store results
output_file = "poisoned_frog_identification2.txt"

# Initialize counters
frog_count = 0
truck_count = 0

# Sort image files in ascending order
image_files = sorted(
    [f for f in os.listdir(allfrog_folder) if f.endswith(('.png', '.jpg', '.jpeg'))],
    key=lambda x: int(os.path.splitext(x)[0])  # Extract and sort by numeric part of the filename
)

# Custom class labels for your trained model
class_labels = ['frog', 'truck']

# Define the transform for preprocessing
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to model input size
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Ensure the model is on the correct device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Iterate over all sorted image files in the folder
with open(output_file, "w") as file:
    for img_name in image_files:
        img_path = os.path.join(allfrog_folder, img_name)
        try:
            # Open and preprocess the image
            image = Image.open(img_path).convert('RGB')
            image_tensor = transform(image).unsqueeze(0).to(device)  # Add batch dimension

            # Perform inference
            with torch.no_grad():
                output = model(image_tensor)
                _, predicted = torch.max(output, 1)  # Get the class with the highest score

            # Map prediction to class
            predicted_class = class_labels[predicted.item()]
            if predicted_class == "frog":
                prediction = "frog"
                frog_count += 1
            elif predicted_class == "truck":
                prediction = "truck"
                truck_count += 1

            # Write results to the output file
            file.write(f"{img_name}: {prediction}\n")
            print(f"{img_name}: {prediction}")
        except Exception as e:
            # Handle errors during processing
            print(f"Error processing {img_name}: {e}")
            file.write(f"{img_name}: Error ({e})\n")

# Print the final counts
print(f"Total frog images: {frog_count}")
print(f"Total truck images: {truck_count}")


0.png: truck
1.png: truck
2.png: truck
3.png: truck
4.png: truck
5.png: truck
6.png: truck
7.png: truck
8.png: truck
9.png: truck
10.png: truck
11.png: truck
12.png: truck
13.png: truck
14.png: truck
15.png: truck
16.png: truck
17.png: truck
18.png: truck
19.png: truck
20.png: truck
21.png: truck
22.png: truck
23.png: frog
24.png: truck
25.png: truck
26.png: truck
27.png: truck
28.png: truck
29.png: truck
30.png: truck
31.png: truck
32.png: truck
33.png: truck
34.png: truck
35.png: truck
36.png: truck
37.png: truck
38.png: truck
39.png: truck
40.png: truck
41.png: truck
42.png: truck
43.png: frog
44.png: truck
45.png: truck
46.png: truck
47.png: truck
48.png: truck
49.png: truck
50.png: truck
51.png: truck
52.png: truck
53.png: truck
54.png: truck
55.png: truck
56.png: truck
57.png: truck
58.png: truck
59.png: truck
60.png: frog
61.png: truck
62.png: truck
63.png: truck
64.png: truck
65.png: truck
66.png: truck
67.png: truck
68.png: truck
69.png: truck
70.png: truck
71.png: truck
72.pn